### 以下部分只需要執行一遍

In [ ]:
# ngrok
import getpass
from pyngrok import ngrok, conf
from flask import Flask, request, abort
from pyngrok import ngrok

# 環境變數
import os
from dotenv import load_dotenv

# 檔案處理
import joblib
import pandas as pd

# LineBot
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, TemplateSendMessage, ButtonsTemplate, PostbackAction, PostbackEvent

In [ ]:
# 和 ngrok 帳戶進行綁定
!ngrok authtoken 2ijB2VvqZjVlxIhjxBtYK5xxWPA_5phxT2XFA6Cuviu4iAmSk

In [ ]:
# 輸入 ngrok token 並與 ngrok 進行認證。
print("Enter your authtoken")
conf.get_default().auth_token = getpass.getpass()

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect("22", "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")

# 2ijB2VvqZjVlxIhjxBtYK5xxWPA_5phxT2XFA6Cuviu4iAmSk

In [ ]:
# 載入 .env 文件
load_dotenv()

In [ ]:
app = Flask(__name__)
port = 5000

# Set up ngrok tunnel to expose local server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\" ") # 印出 tunnel

# Line API 驗證
access_token = os.getenv('LINE_ACCESS_TOKEN')
secret = os.getenv('LINE_SECRET')
line_bot_api = LineBotApi(access_token)  # token 確認
handler = WebhookHandler(secret)      # secret 確認

In [ ]:
"""
接收並處理來自 Line 平台的 Webhook 請求。
獲取並驗證請求的簽名。
調用相應的處理函數處理請求數據。
在簽名驗證失敗時返回 400 錯誤碼。
"""
@app.route("/", methods=['POST'])
def webhook():
    body = request.get_data(as_text=True)
    signature = request.headers['X-Line-Signature']
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

### 每次都要執行一遍

In [ ]:
# 載入模型
model = joblib.load('./diabete_prediction_model.pkl')

In [ ]:
# 定義 Question 類別, 方便問問題
class Question:
    def __init__(self, question_text):
        # 設定問題的文字內容
        self.question_text = question_text
    def ask_question(self, reply_token):
        raise NotImplementedError("這個方法應該在子類別實現")
    
class TextQuestion(Question):
    def __init__(self, question_text):
        # 初始化跟父類別相同
        super().__init__(question_text)

    def ask_question(self, reply_token):
        # 傳送文字問題
        message = TextSendMessage(text=self.question_text)
        # 使用 reply 方法傳送
        line_bot_api.reply_message(reply_token, message)

class ButtonQuestion(Question):
    def __init__(self, question_text, choices, introduction=None):
        super().__init__(question_text)
        self.introduction = introduction if introduction else "請選擇您的" + question_text
        self.choices = choices

    def ask_question(self, reply_token):
        actions = [PostbackAction(label=label, data=data) for label, data in self.choices]
        template_message = TemplateSendMessage(
            alt_text= "請輸入" + self.question_text,
            template=ButtonsTemplate(
                title=self.question_text,
                text=self.introduction,
                actions=actions
            )
        )
        line_bot_api.reply_message(reply_token, template_message)

In [ ]:
# 初始化新的使用者
def initializeNewUser(reply_token, user_id):
    # 在函數當中使用全域變數, 需要告知 python
    global user_state 
    global questions
    # 初始化
    user_state[user_id] = UserState(questions)
    # 問第一個問題
    user_state[user_id].questions[0].ask_question(reply_token)
    return

In [ ]:
class UserState:
    # 建構函式, 初始化用戶狀態, 紀錄 step, Question, data
    def __init__(self, questions):
        self.step = 0
        self.questions = questions
        self.data = []

In [28]:
# 紀錄用戶當前輸入狀態
user_state = {}
# 問題列表, 之後可以在更加簡化
introduction = ("您好，我是健康智能管家。\n"
                "也是糖尿病預測專家，\n"
                "您可以叫我阿瑄=U=\n"
                "請問是否進行糖尿病預測呢?")
introQuestion = ButtonQuestion('是否進行糖尿病預測？', [('是', 'continue'), ('否', 'exit')], introduction)
diabeteQuestions = [
    ButtonQuestion('性別', [('男', '0'), ('女', '1'), ('其他', '2')]),
    ButtonQuestion('高血壓狀況', [('有', '1'), ('無', '0')]),
    ButtonQuestion('心臟病狀況', [('有', '1'), ('無', '0')]),
    ButtonQuestion('吸煙習慣', [('天天抽菸', '4'), ('目前有吸菸', '3'), ('曾經有過', '1'), ('未曾吸菸', '0')]),
    TextQuestion("請輸入年齡: "),
    TextQuestion("請輸入BMI: "),
    TextQuestion("請輸入HbA1c水平: "),
    TextQuestion("請輸入血糖水平: ")
]
questions = [introQuestion, *diabeteQuestions]



In [ ]:
# 紀錄用戶當前輸入狀態
user_state = {}
# 問題列表, 之後可以在更加簡化
introduction = ("您好，我是健康智能管家。\n"
                "也是糖尿病預測專家，\n"
                "您可以叫我阿瑄=U=\n"
                "請問是否進行糖尿病預測呢?")
introQuestion = ButtonQuestion('是否進行糖尿病預測？', [('是', 'continue'), ('否', 'exit')], introduction)
diabeteQuestions = [
    ButtonQuestion('性別', [('男', '0'), ('女', '1'), ('其他', '2')]),
    ButtonQuestion('高血壓狀況', [('有', '1'), ('無', '0')]),
    ButtonQuestion('心臟病狀況', [('有', '1'), ('無', '0')]),
    ButtonQuestion('吸煙習慣', [('天天抽菸', '4'), ('目前有吸菸', '3'), ('曾經有過', '1'), ('未曾吸菸', '0')]),
    TextQuestion("請輸入年齡: "),
    TextQuestion("請輸入BMI: "),
    TextQuestion("請輸入HbA1c水平: "),
    TextQuestion("請輸入血糖水平: ")
]
questions = [introQuestion, *diabeteQuestions]

In [ ]:
# 最後輸出, 根據使用者輸入預測結果
def process_final_input(reply_token, userState):
    # 獲取使用者資料
    user_data = userState.data
    # pop 第一個問題的答案 ( 詢問是否預測 )
    user_data.pop(0)
    # 將使用者輸入轉換成 pandas, 並加入標籤
    user_input = pd.DataFrame([user_data], columns=[
        'gender', 'age', 'hypertension', 'heart_disease', 'smoking_history',
        'bmi', 'HbA1c_level', 'blood_glucose_level'
    ])

    prediction = model.predict(user_input)
    result = "没有糖尿病" if prediction[0] == 0 else "有糖尿病"

    prediction = model.predict_proba(user_input)[0]

    # reply Message, 接收用戶輸入之後傳送結果
    line_bot_api.reply_message(reply_token, [
        TextSendMessage(text=f"{result}"),
        TextSendMessage(text=f"糖尿病機率:{prediction[1]*100:.2f}%"),
    ])   
    EndPrediction(reply_token, userSession)
# 刪除使用者資料
def EndPrediction(reply_token, user_id):
    line_bot_api.reply_message(reply_token, TextSendMessage(text="謝謝光臨!! 有需要都可以在叫我喔")) 
    del user_state[user_id]

In [ ]:
# 正整數驗證
def validate_numeric_input(event, msg):
    if not msg.isdigit():
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="請輸入正確的數字"))
        return False
    if float(msg) <= 0:
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="請輸入大於 0 的有效數字"))
        return False
    return True

In [ ]:
#下一個問題
def NextQuestion(reply_token, user_id):
    user_state[user_id]['step'] += 1
    # 還沒到最後一個問題: 繼續問下一個問題
    if user_state[user_id]['step'] < len(questions):
        questions[user_state[user_id]['step']].ask_question(reply_token)
    else:
        # 否則輸出最後結果
        process_final_input(reply_token, user_id)

In [ ]:
# 用戶傳送訊息的時候做出的回覆
@handler.add(MessageEvent, message=TextMessage)
def handle_text_message(event):
    user_id = event.source.user_id
    msg = event.message.text
    # 初始化新的使用者 or 判斷輸入
    if user_id not in user_state:
        initializeNewUser(event.reply_token, user_id)
        return
    if isinstance(questions[user_state[user_id]['step']], ButtonQuestion):
        # 按鈕問題不應該輸入文字回答
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="請選擇按鈕選項"))
        return
    # 數字驗證
    if not validate_numeric_input(event, msg):
        return
    # 將資料加入, 前往下一題
    # 驗證通過，加入正確資料，前往下一題目
    user_state[user_id]['data'].append(float(msg))
    NextQuestion(event.reply_token, user_id)

In [ ]:
# 按鈕按下之後的回應
@handler.add(PostbackEvent)
def handle_postback(event):
    # 獲取使用者與回傳的按鈕資訊
    user_id = event.source.user_id
    postback_data = event.postback.data
    # 初始化新的使用者 or 判斷輸入
    if user_id not in user_state:
        initializeNewUser(event.reply_token, user_id)
        return
    elif isinstance(questions[user_state[user_id]['step']], TextQuestion):
        # 文字問題不接受按鈕回答
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="請輸入數字"))
        return
    # 第一個的按鈕問題特別判斷 ( 任何問題都可以加入 exit)
    user_state[user_id]['data'].append(postback_data)
    if(postback_data == 'exit'):
        EndPrediction(event.reply_token, user_id)
        return
    NextQuestion(event.reply_token, user_id)

In [ ]:
print(public_url)
if __name__ == "__main__":
    app.run(port=port)
# 2ijB2VvqZjVlxIhjxBtYK5xxWPA_5phxT2XFA6Cuviu4iAmSk
# tasklist /FI "IMAGENAME eq ngrok.exe
# taskkill /PID ngrok.exe /F